In [1]:
# 将数据载入内存后，按query划分
# 而对每个query内数据随机抽取一半，比对另一半。如果playtime比另一半小，则为逆序对
# 对逆序对，第一轮时一个拟合1，一个-1（即deta取1）。拟合出一棵树后，再对每个query内数据随机抽取一半
# 进行下一轮训练。此时按公式先获得先前预测值，而后如果存在逆序对，再按学习率为1的公式进行更新该对目标函数
# 从而获取该对的负梯度方向，再构造树进行拟合

from IPython.display import Image  
from sklearn import tree
import pydotplus 

import numpy as np
import joblib
from sklearn.tree import DecisionTreeRegressor
import time

In [1]:
query_X = {}

f = open('train_data','r',encoding='utf-8')
lines = f.readlines()
for _ind,_line in enumerate(lines):
    if _ind%1000==0:
        print(_ind)
    _line = _line[:-1]
    query,gameid,features = _line.split('\t',2)
    if query not in query_X.keys():
        query_X[query] = {}
    query_X[query][gameid] = [float(x) for x in features.split('\t')]
f.close()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000


In [2]:
# 去掉query_X里query只有一条数据的，还有query里的数据时长都是0的
queryCantTrain = set()
for query,gameid_feat in query_X.items():
    if len(gameid_feat)==1:
        queryCantTrain.add(query)
        continue
    playTimeAllZeroFlag = True
    for gameid,feat in gameid_feat.items():
        if feat[0]!=0:
            playTimeAllZeroFlag = False
    if playTimeAllZeroFlag:
        queryCantTrain.add(query)
        continue
        
for query in queryCantTrain:
    del query_X[query]
    

In [56]:
# 获取排序标准：游戏时长
query_playtime = {}

for query,gameid_feat in query_X.items():
    for gameid,feat in gameid_feat.items():
        if query not in query_playtime.keys():
            query_playtime[query] = {}
        query_playtime[query][gameid] = feat[0]
        

In [57]:
# 将数据拆分为训练集和验证集
# 一万个query数据做训练，剩下的验证

data_cont = len(query_X)
train_cont = 10000
vali_cont = data_cont-train_cont

permut = np.random.permutation(data_cont)

query_X_train = {}
query_X_vali = {}

query_X_list = list(query_X.items())
for _ind in range(0,data_cont):
    if _ind<train_cont:
        query_X_train[query_X_list[permut[_ind]][0]] = query_X_list[permut[_ind]][1]
    else:
        query_X_vali[query_X_list[permut[_ind]][0]] = query_X_list[permut[_ind]][1]

In [58]:
# ndcg得分算法
# 由于我们只关注序（训练时也是），那我们scores就拿序输入就行，不用拿时间
# 我们最关注的还是不能把该保留的丢到后面取
def getDCG(scores):
    scores = np.array(scores)
    return np.sum(
        # np.divide(np.power(2, scores) - 1, np.log(np.arange(scores.shape[0], dtype=np.float32) + 2)),
        np.divide(scores, np.log(np.arange(scores.shape[0], dtype=np.float32) + 2)),
        dtype=np.float32)


def getNDCG(cur_rank):
    best_rank = np.sort(cur_rank)[::-1]
    worst_rank = np.sort(cur_rank)
    
    idcg = getDCG(best_rank)
    wdcg = getDCG(worst_rank)
    dcg = getDCG(cur_rank)

    # print(dcg,idcg,wdcg)
    if idcg-wdcg!=0:
        ndcg = (dcg-wdcg) / (idcg-wdcg)
    else:
        ndcg = 1
    return ndcg


cur_rank = [1, 2,1,3]
print(getNDCG(cur_rank))


0.10312879


In [93]:
# 训练并验证的主要函数


def fit(sampled_X,sampled_y,max_depth):
    global regrs
    print('start fit')
    regr = DecisionTreeRegressor(max_depth=max_depth)
    regr.fit(sampled_X, sampled_y)
    regrs.append(regr)
    return regr

def getNextScore(cur_scores,regri,query_X,k):
    # 叠加公式为 (k*score_ori+beta*score_curgradient)/(k+1)，其中k表示第几次梯度拟合了
    # 整个公式表示越拟合到后面，当前梯度对全局影响越小。是合理的
    print('start getNextScore ',k)
    beta = 1
    X = []
    ind = 0
    indToQueryGameid = {}
    for query,gameid_feat in query_X.items():
        for gameid,feat in gameid_feat.items():
            X.append(feat[1:])
            indToQueryGameid[ind] = query+'_'+gameid
            ind += 1
    tmp = time.time()
    scores = regri.predict(X)
    deta_time = time.time()-tmp
    print('predict time: ',deta_time)
    next_scores = {}
    for i in range(0,len(scores)):
        tmp = indToQueryGameid[i].split('_')
        gameid = tmp[-1]
        query = '_'.join(tmp[:-1])
        # query,gameid = indToQueryGameid[ind].split('_')
        if query not in next_scores.keys():
            next_scores[query] = {}
        next_scores[query][gameid] = (k*cur_scores[query][gameid] + beta*scores[i])/(k+1)
    return next_scores 

def sampleQueryRevPairsAndGeneLoss(cur_scores,query_X): 
    # cur_scores为：{'query1':{'gameid1':0.1,'gameid2':0.3,...},...}
    # 取最小间隔为1.先在每个query内permut，然后两个两个取获得pairs,
    # 然后playtime(pair[0])>playtime(pair[1])且score(pair[0])<score(pair[1])+1的即为逆序对，
    # pair[0]的梯度方向为score(pair[1])+1-score(pair[0])
    # pair[1]的梯度方向为score(pair[0])-1-score(pair[1])
    # 然后既可以获得下一棵回归树的目标函数
    print('start sampleQueryRevPairsAndGeneLoss ')
    deta = 1
    sampled_X = []
    sampled_y = []
    for query,gameid_feat in query_X.items():
        tmp = dict([[ind,(gameid,feat)] for ind,(gameid,feat) in enumerate(gameid_feat.items())])
        permut = np.random.permutation(len(tmp))
        for i in range(0,len(tmp),2):
            if i+1<len(tmp):
                score0 = cur_scores[query][tmp[permut[i]][0]]
                score1 = cur_scores[query][tmp[permut[i+1]][0]]
                playtime0 = tmp[permut[i]][1][0]
                playtime1 = tmp[permut[i+1]][1][0]
                feat0 = tmp[permut[i]][1][1:]
                feat1 = tmp[permut[i+1]][1][1:]
                if playtime0 > playtime1 and score0 < score1 + deta:
                    # sampled_X.append(feat0+[score1+deta-score0])
                    # sampled_X.append(feat1+[score0-deta-score1])
                    sampled_X.append(feat0)
                    sampled_X.append(feat1)
                    sampled_y.append(score1+deta-score0)
                    sampled_y.append(score0-deta-score1)
    return sampled_X,sampled_y

In [60]:
# 在本项目中获取ndcg得分的办法
def getNdcgScore(playtime_dict,predscore_dict):
    playtime_list_asc = sorted(list(playtime_dict.items()),key=lambda x:x[1])
    gameid_to_truerank = {}
    ind = 1
    last_playtime = -1
    for itm in playtime_list_asc:
        if last_playtime == -1:
            gameid_to_truerank[itm[0]] = ind
            last_playtime = itm[1]
        elif itm[1] == last_playtime:
            gameid_to_truerank[itm[0]] = ind
        else:
            ind += 1
            gameid_to_truerank[itm[0]] = ind
            last_playtime = itm[1]
    # print(gameid_to_truerank)
    score_list_desc = sorted(list(predscore_dict.items()),key=lambda x:-x[1])
    dest_rnk = []
    for itm in score_list_desc:
        dest_rnk.append(gameid_to_truerank[itm[0]])
    return getNDCG(dest_rnk)

ndcg_scores_dict = {}
def getAvgNdcgScore(cur_scores,query_playtime):
    global ndcg_scores
    for query,gameid_score in cur_scores.items():
        ndcg_score = getNdcgScore(query_playtime[query],gameid_score)
        ndcg_scores_dict[query] = ndcg_score
        # ndcg_scores.append(ndcg_score)
    return np.average(list(ndcg_scores_dict.values()))

# playtime_dict = query_playtime['迷你世界']
# predscore_dict = cur_scores['迷你世界']
# print(getNdcgScore(playtime_dict,predscore_dict))
# print(getAvgNdcgScore(cur_scores,query_playtime))
# print([[k,v] for k,v in ndcg_scores_dict.items() if np.isnan(v)])

In [94]:
# 训练主程序

# 初始化得分
cur_scores_train = {}
for query,gameid_feat in query_X_train.items():
    for gameid,feat in gameid_feat.items():
        if query not in cur_scores_train.keys():
            cur_scores_train[query] = {}
        cur_scores_train[query][gameid] = 0

cur_scores_vali = {}
for query,gameid_feat in query_X_vali.items():
    for gameid,feat in gameid_feat.items():
        if query not in cur_scores_vali.keys():
            cur_scores_vali[query] = {}
        cur_scores_vali[query][gameid] = 0
        
# 要训练几次

regrs = []
train_times = 8
tree_max_deep = 8
for i in range(0,train_times):
    sampled_X,sampled_y = sampleQueryRevPairsAndGeneLoss(cur_scores_train,query_X_train)
    regri = fit(sampled_X,sampled_y,tree_max_deep)
    cur_scores_train = getNextScore(cur_scores_train,regri,query_X_train,i)
    print('cur_scores_train score: ',getAvgNdcgScore(cur_scores_train,query_playtime),' epoch: ', i)
    cur_scores_vali = getNextScore(cur_scores_vali,regri,query_X_vali,i)
    print('cur_scores_vali score: ',getAvgNdcgScore(cur_scores_vali,query_playtime),' epoch: ',i)

start sampleQueryRevPairsAndGeneLoss 
start fit
start getNextScore  0
predict time:  0.9933452606201172
cur_scores_train score:  0.6819416966672734  epoch:  0
start getNextScore  0
predict time:  0.18650126457214355
cur_scores_vali score:  0.6716843945067161  epoch:  0
start sampleQueryRevPairsAndGeneLoss 
start fit
start getNextScore  1
predict time:  1.000326156616211
cur_scores_train score:  0.6853225446724814  epoch:  1
start getNextScore  1
predict time:  0.17453289031982422
cur_scores_vali score:  0.6854013902798592  epoch:  1
start sampleQueryRevPairsAndGeneLoss 
start fit
start getNextScore  2
predict time:  1.0761241912841797
cur_scores_train score:  0.7004322680978541  epoch:  2
start getNextScore  2
predict time:  0.18151426315307617
cur_scores_vali score:  0.7017907402168833  epoch:  2
start sampleQueryRevPairsAndGeneLoss 
start fit
start getNextScore  3
predict time:  1.019275426864624
cur_scores_train score:  0.7153344523108941  epoch:  3
start getNextScore  3
predict tim

In [95]:
# 将回归器存储

for i in range(0,train_times):
    joblib.dump(regrs[i], 'gbdt_model_epoch_'+str(i)+'.m')   # 保存模型


In [96]:
# 载入回归器
regrs_predict = []
for i in range(0,train_times):
    regrs_predict.append(joblib.load('gbdt_model_epoch_'+str(i)+'.m'))
    

In [97]:
# 预测(选一个多item的query)
X_ind = query_X['迷你世界'].keys()
X = [x[1:] for x in query_X['迷你世界'].values()]


def getScorePredict(X,regrs,beta):
    # 叠加公式为 (k*score_ori+beta*score_curgradient)/(k+1)，其中k表示第几次梯度拟合了
    # 整个公式表示越拟合到后面，当前梯度对全局影响越小。是合理的
    scores = None
    tmp = time.time()
    for k,regri in enumerate(regrs):
        if k == 0:
            scores = beta*regri.predict(X)
        else:
            scores = (k*scores + beta*regri.predict(X))/(k+1)
    deta_time = time.time()-tmp
    print('predict time: ',deta_time)
    return scores 


scores = getScorePredict(X,regrs_predict,1)

print('cur_scores_vali score: ',getNdcgScore(query_playtime['迷你世界'],dict(zip(X_ind,scores))))


predict time:  0.06283140182495117
cur_scores_vali score:  0.8724221


In [98]:
# 找个比较长的做性能测试
tmp = sorted(list(query_X.items()),key=lambda x:-len(x[1]))
len(tmp[0][1])

# 将query为 迷你世界 和 迷你器 的游戏放一起来模仿粗排长度
X_ind = list(query_X['迷你世界'].keys())
X = [x[1:] for x in query_X['迷你世界'].values()]
X_playtime = [x[0] for x in query_X['迷你世界'].values()]

for gameid,feat in query_X['模拟器'].items():
    if gameid not in X_ind:
        X_ind.append(gameid)
        X.append(feat[1:])
        X_playtime.append(feat[0])

scores = getScorePredict(X,regrs_predict,1)

print('cur_scores_vali score: ',getNdcgScore(dict(zip(X_ind,X_playtime)),dict(zip(X_ind,scores))))



predict time:  0.09075713157653809
cur_scores_vali score:  0.82894444


In [2]:
# 查看节点分裂情况

train_times = 8
regrs_predict = []
for i in range(0,train_times):
    regrs_predict.append(joblib.load('gbdt_model_epoch_'+str(i)+'.m'))
    
for i in range(0,train_times):
    dot_data = tree.export_graphviz(regrs_predict[i], out_file=None,  #regr_1 是对应分类器
                             feature_names=[str(x) for x in list(np.arange(0,770))],   #对应特征的名字
                             class_names=['y'],    #对应类别的名字
                             filled=True, rounded=True,  
                             special_characters=True)  
    graph = pydotplus.graph_from_dot_data(dot_data)  
    graph.write_png('tree_cut_%d.png'%i)    #保存图像
    Image(graph.create_png()) 


dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.911613 to fit

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.911613 to fit

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.983226 to fit

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.983226 to fit

